In [ ]:
import torch
import transformers
import pandas as pd
import numpy as np
import re
import gc
import pickle

data = pd.read_csv("data_all.csv")

In [ ]:
texts = data["text"].str.replace("[^А-я]", " ", regex=True).str.lower()

In [ ]:
seq = None
inputs = None
model = None
tokenizer = None
out = None
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_reserved()

In [ ]:
list_of_arrays = []

In [ ]:
model_name = "DeepPavlov/rubert-base-cased"
model = transformers.AutoModel.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
for i in range(data.shape[0] // 100 + 1):
    with torch.no_grad():
        seq = list(texts.values[i * 100 : (i + 1) * 100])
        inputs = tokenizer(
            seq,
            return_tensors="pt",
            pad_to_max_length=True,
            max_length=128,
            truncation=True,
        ).to(device)
        out = model(**inputs)[0]
        out_np = out.detach().cpu().numpy()
        for arr in out_np:
            list_of_arrays.append(arr)
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
arr = np.array([x.reshape((-1)) for x in list_of_arrays])

In [ ]:
data_encoded = pd.DataFrame(arr, columns=list(range(arr.shape[1])))

In [ ]:
with open("embeddings", "wb") as encodings:
    pickle.dump(data_encoded, encodings)